In [6]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [8]:
# export env TAVILY_API_KEY from https://tavily.com/ as researcher to use this tool
import os
os.environ['TAVILY_API_KEY'] = "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"

In [14]:
tool = TavilySearchResults(max_results=2)
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [11]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [13]:
class Agent:
  def __init__(self, model, tools, system=""):
    self.system = system
    graph = StateGraph(AgentState)
    graph.add_node("llm", self.call_openai)
    graph.add_node("action", self.take_action)
    graph.add_conditional_edges(
      "llm",
      self.exists_action,
      {True: "action", False: END}
    )
    graph.add_edge("action", "llm")
    graph.set_entry_point("llm")
    self.graph = graph.compile()
    self.tools = {t.name: t for t in tools}
    self.model = model.bind_tools(tools)

def exists_action(self, state: AgentState):
  result = state['messages'][-1]
  return len(result.tool_calls) > 0

def call_openai(self, state: AgentState):
  messages = state['messages']
  if self.system:
    messages = [SystemMessage(content=self.system)] + messages
  message = self.model.invoke(messages)
  return { 'messages': [message]}

def take_action(self, state: AgentState):
  tool_calls = state['messages'][-1].tool_calls
  results = []
  for t in tool_calls:
    print(f"Calling: {t}")
    if not t['name'] in self.tools:
      print("\n ... bad tool name...")
      result = "bad tool name, retry"
    else:
      result = self.tools[t['name']].invoke(t['args'])
    results.append(ToolMessage(tool_call_id=t['id']), content=str(result))
    print("Back to the mdoel")
    return { 'messages': results}